<a href="https://colab.research.google.com/github/M-torki/ECG-Classification/blob/main/MIT_BIH_Ozal_N_L_R_V_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Main Dataset used in this study is MIT-BIH Arrhythmia database from PhysioNet:
[This](https://physionet.org/content/mitdb/1.0.0/) 
is a link to physionet MIT-BIH.

Arrhythmia classes are prepared according to Ozal Yildirim's in this NoteBook.

link to related papers:
[Ozal Paper](https://sci-hub.do/https://www.sciencedirect.com/science/article/abs/pii/S0010482518300738) 
&
[Ozal et al. paper](https://sci-hub.do/https://www.sciencedirect.com/science/article/abs/pii/S0169260718314329)

Arryhthmia calsses: 

L: Left bundle branch block 

R: Right bundle branch block

V: Premature ventricular contraction

PB(/): Paced beat

N: Normal



Test and train sets are seperated according to patients (ie. not just beats).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wfdb

     |████████████████████████████████| 133kB 10.8MB/s 


In [ ]:
! pip install biosppy

     |████████████████████████████████| 92kB 5.0MB/s 


In [ ]:
! pip install neurokit2

     |████████████████████████████████| 993kB 10.8MB/s 


In [ ]:
import os
import numpy as np
import wfdb
import pywt
import matplotlib.pyplot as plt
import pandas as pd
import biosppy
import scipy
import neurokit2 as nk
from wfdb.processing import resample_singlechan
from wfdb.plot import plot_items
from wfdb.processing import normalize_bound
from scipy.signal import find_peaks
from scipy.signal import resample

from wfdb import Annotation
from wfdb import show_ann_labels
from wfdb.processing import find_local_peaks

In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
cd mit-bih-arrhythmia-database-1.0.0

/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0


In [ ]:
ls

100.atr    108.at_  117.at_  200.xws  210.dat  221.atr  234.atr
100.dat    108.atr  117.atr  201.atr  210.hea  221.dat  234.dat
100.hea    108.dat  117.dat  201.dat  210.xws  221.hea  234.hea
100.xws    108.hea  117.hea  201.hea  212.atr  221.xws  234.xws
101.atr    108.xws  117.xws  201.xws  212.dat  222.at_  ANNOTATORS
101.dat    109.atr  118.atr  202.atr  212.hea  222.atr  a_train.txt
101.hea    109.dat  118.dat  202.dat  212.xws  222.dat  A_train.txt
101.xws    109.hea  118.hea  202.hea  213.atr  222.hea  best_model.h5
102-0.atr  109.xws  118.xws  202.xws  213.dat  222.xws  F_train.txt
102.atr    111.atr  119.at_  203.at_  213.hea  223.atr  J_train.txt
102.dat    111.dat  119.atr  203.at-  213.xws  223.dat  L_train.txt
102.hea    111.hea  119.dat  203.atr  214.at_  223.hea  MIT-BIH/
102.xws    111.xws  119.hea  203.dat  214.atr  223.xws  MIT_BIH_Record_100.csv
103.atr    112.atr  119.xws  203.hea  214.dat  228.atr  MITBIHtest1.csv
103.dat    112.dat  121.atr  203.xws  214.hea  228.

In [ ]:
df = pd.read_csv("./RECORDS", header=None)
df.head()

,0
0,100
1,101
2,102
3,103
4,104


##reading signals and their annotation files (lead II) for 48 patients

In [ ]:
sig = list()
ann = list()


for i in range (48):
    if i != 13:
        sig.append(wfdb.rdsamp(str(df[0][i]), channels=[0])[0])
    if i == 13:
        sig.append(wfdb.rdsamp(str(df[0][i]), channels=[1])[0])    
    ann.append(wfdb.rdann(str(df[0][i]),'atr',summarize_labels=True))


##we keep the sample rate equal to 360 Hz

In [ ]:
resig = sig
reann = ann

In [ ]:
resig = np.reshape(resig, (48,650000))

##the number of samples in 10 seconds segment

In [ ]:
samples = 360*10
samples

3600

##each signal contains 180 number of 10 seconds segments

In [ ]:
648000/3600

180.0

In [ ]:
seg = list()

for i in range (len(resig)):
    seg.append(np.split(resig[i][0:648000], 3600))

seg = np.asarray(seg)
seg = np.reshape(seg, (48,180,3600))
np.shape(seg)  

(48, 180, 3600)

##10 seconds segmented signal annotation

In [ ]:
an = np.empty((48,180), dtype='object')

for j in range (48):
    for k in range (180):
        a = list()
        for i in range (len(reann[j].sample)):
            if ((k*3600) < reann[j].sample[i] < ((k+1)*3600)):
                a.append(((reann[j].sample[i]-(k*3600)),reann[j].symbol[i])) 
        an[j][k] = a

##normalizing each segment between 0 and 1

In [ ]:
norm = np.empty_like(seg)

for i in range (len(seg)):
    for j in range(180):
        norm[i,j] = normalize_bound(seg[i,j],0,1)

norm = np.asarray(norm)
np.shape(norm)

(48, 180, 3600)

##finding the location of each peak using annotation sample

In [ ]:
peaks = np.empty((48,180), dtype='object')
for i in range (48):
    for j in range (180):
        a = list()
        for k in range (len(an[i,j])):
            a.append(an[i,j][k][0])
        peaks[i,j] = a            

##finding the label of each peak using annotation symbol

In [ ]:
label = np.empty((48,180), dtype='object')

for i in range (48):
    for j in range (180):
        a = list()
        for k in range (len(an[i,j])):
            a.append(an[i,j][k][1])
        label[i,j] = a                    

##Segments were created to include 99 samples prior to the R peak, and 160 samples following the R peak

In [ ]:
data = np.empty((48,180), dtype='object')

for i in range (48):
    for j in range (180):
        peak = peaks[i,j]
        h = list()
        for k in range (len(peak)):
            start , end = (peaks[i][j][k]-99), (peaks[i][j][k]+160)
            if (start<0):
                start = 0
            if (end>3600):
                end = 3600
            e1 = norm[i][j][start:end]    
            if (len(e1) == 0):
                print('an empty array!')  
            if (len(e1)==259):
                h.append(e1)  
                 
            if (len(e1)<259):
                e = np.pad(e1,(0,259-len(e1)), mode = 'constant')
                h.append(e)
                
            if (len(e1)>259):
                e = resample(e1, num = 259)
                h.append(e)
                 

             
                         
        data[i,j] = h

##checking if their length are equal

In [ ]:
for i in range (48):
    for j in range (180):
        peak = peaks[i,j]
        data1 = data[i][j]
        for k in range (len(peak)):
            if (len(data1[k])!= 259):
                print('noo!')


In [ ]:
cd -

/content/drive/MyDrive


In [ ]:
cd mit-bih-arrhythmia-database-1.0.0

/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0


In [ ]:
mkdir MIT-BIH1

In [ ]:
cd MIT-BIH1/

/content/drive/My Drive/mit-bih-arrhythmia-database-1.0.0/MIT-BIH


##saving beats from specific patient as a dataframe for later use

In [ ]:
for i in range (48):
    for j in range (180):
        dataframe0 = pd.DataFrame(data[i][j], columns=[x for x in range (259)])
        dataframe1 = pd.DataFrame(label[i][j], columns=['Label']) 
        dataframe2 = pd.concat([dataframe0, dataframe1], axis = 1, ignore_index=True)
        dataframe3 = dataframe2.dropna(axis=0, how='any')
        dataframe3.to_csv('dataframe{},{}.csv'.format(i,j))       

##reading mentioned beats

In [ ]:
import pandas as pd
import glob

path = r'./' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, ignore_index=True)

In [ ]:
df1 = frame
df1 = df1.drop(labels=['Unnamed: 0'], axis=1)

##the number of beats with mentioned label

In [ ]:
labels = df1['259']
equilibre = labels.value_counts()
print(equilibre)

N    74785
L     8053
R     7237
V     7112
/     7003
A     2530
+     1288
f      979
F      802
~      612
!      471
"      437
j      229
x      192
a      149
|      132
E      105
J       83
Q       33
e       16
]        6
[        6
S        2
Name: 259, dtype: int64


##keeping normal, left and right bundle brunch, Premature ventricular contraction and paced beats

In [ ]:
df1 = df1.replace({'259': {'[': np.nan, ']': np.nan, 'S': np.nan, 'e': np.nan,
                           'Q': np.nan, 'J': np.nan, 'E': np.nan, '|': np.nan, 'a': np.nan, 'x': np.nan, 'j': np.nan,
                           '"': np.nan, '!': np.nan, '~': np.nan, 'F': np.nan, 'f': np.nan, '+': np.nan, 'A': np.nan}})
df1 = df1.dropna()
labels = df1['259']
equilibre = labels.value_counts()
print(equilibre)

N    74785
L     8053
R     7237
V     7112
/     7003
Name: 259, dtype: int64


In [ ]:
df1 = df1.replace({'259': {'N': 0, 'R': 1, 'L': 2, 'V': 3, '/': 4}})

In [ ]:
labels = df1['259']
equilibre = labels.value_counts()
print(equilibre)

0    74785
2     8053
1     7237
3     7112
4     7003
Name: 259, dtype: int64


##spliting test and train datasets with 20% ratio

In [ ]:
n = df1[df1['259'] == 0]
test0 = n[:14957]
train0 = n[14957:]



In [ ]:
r = df1[df1['259'] == 1]
test1 = r[:1446]
train1 = r[1446:]


In [ ]:
v = df1[df1['259'] == 3]
test2 = v[:1421]
train2 = v[1421:]


In [ ]:
a = df1[df1['259'] == 4]
test3 = a[:1400]
train3 = a[1400:]


In [ ]:
l = df1[df1['259'] == 2]
test4 = l[:1610]
train4 = l[1610:]


In [ ]:
cd -

/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0


##saving test dataset to csv

In [ ]:
test = pd.concat([test0, test1, test2, test3, test4], ignore_index=True)
test.to_csv('MITBIHtest2.csv')

##saving train dataset to csv

In [ ]:
train = pd.concat([train0, train1, train2, train3, train4], ignore_index=True)
train.to_csv('MITBIHtrain2.csv')